In [1]:

config_path = r'E:\编程\CODE\MMLAB自动化\训练权重和配置文件\ulcer_tissue_convnextv2_RFAinout_Dys_mask2former\hpc06061157_RFAinout_Dys_utp__mask2former_convnextv2-l.py'
checkpoint_path = r'E:\编程\CODE\MMLAB自动化\训练权重和配置文件\ulcer_tissue_convnextv2_RFAinout_Dys_mask2former\top_mIoU_97.7800_iter_3000.pth'



In [2]:
from mmseg.apis import init_model, inference_model

e:\Environments\mmsegmentation_exp\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
import os
import pickle
from mmseg.apis import init_model, inference_model
import torch
from PIL import Image
import numpy as np

def get_image_paths_from_folder(folder_path):
    """
    获取并返回文件夹下所有图片的路径列表。

    参数:
    - folder_path: 文件夹路径

    返回值:
    - 图片路径列表
    """
    # 支持的图片扩展名
    supported_extensions = ('.jpg', '.jpeg', '.png', '.bmp')

    # 获取文件夹下所有文件的路径
    all_files = [os.path.join(folder_path, f) for f in os.listdir(folder_path)]
    
    # 过滤出图片文件
    image_paths = [f for f in all_files if os.path.isfile(f) and f.lower().endswith(supported_extensions)]
    
    return image_paths

def infer_and_save(config_path, checkpoint_path, img_path, save_path, save_result=True, infer_one_by_one=False):
    """
    使用模型加载权重推理img_path所有的图像，并将推理结果保存为.png图片于save_path路径下。
    
    参数:
    - config_path: 模型配置文件路径
    - checkpoint_path: 模型权重文件路径
    - img_path: 图像路径，可以是单个图像路径，也可以是图像路径的列表
    - save_path: 将保存的.png格式图片保存的位置
    - save_result: 默认为True，将推理的得到的result对象保存为.pkl文件于save_path路径下
    - infer_one_by_one: 默认为False，是否逐个推理路径列表中的每个路径
    
    返回值:
    - 推理的Result对象列表或单个Result对象
    """
    
    # 检查img_path是否为列表，如果不是，则转换为列表
    if not isinstance(img_path, list):
        img_path = [img_path]
    
    # 初始化模型
    model = init_model(config_path, checkpoint_path)
    
    results = []
    
    if infer_one_by_one:
        for img in img_path:
            # 推理单个图像
            result = inference_model(model, img)
            results.append(result)
    else:
        # 推理所有图像
        results = inference_model(model, img_path)
    
    # 检查结果是否为单个对象，如果是，则转换为列表
    if not isinstance(results, list):
        results = [results]
    
    for img, result in zip(img_path, results):
        # 将推理结果的张量转换为NumPy数组
        cpu_tensor = result.pred_sem_seg.data.cpu()
        numpy_array = cpu_tensor.numpy()
        
        # 将NumPy数组转换为PIL图像
        image = Image.fromarray(numpy_array[0, :, :].astype(np.uint8) * 255)
        
        # 获取原图像名并去除扩展名
        img_name = os.path.basename(img).rsplit('.', 1)[0]
        save_img_path = os.path.join(save_path, f"{img_name}.png")
        
        # 保存图像为.png格式
        image.save(save_img_path)
        print(f"Segmentation result saved as {save_img_path}")
    
    if save_result:
        # 将结果保存为.pkl文件
        results_pkl_path = os.path.join(save_path, 'results.pkl')
        with open(results_pkl_path, 'wb') as f:
            pickle.dump(results, f)
        print(f"Results object saved as {results_pkl_path}")
    
    return results



In [35]:
import os
import pickle
from mmseg.apis import init_model, inference_model
import torch
from PIL import Image
import numpy as np

def get_image_paths_from_folder(folder_path):
    """
    获取并返回文件夹下所有图片的路径列表。

    参数:
    - folder_path: 文件夹路径

    返回值:
    - 图片路径列表
    """
    # 支持的图片扩展名
    supported_extensions = ('.jpg', '.jpeg', '.png', '.bmp')

    # 获取文件夹下所有文件的路径
    all_files = [os.path.join(folder_path, f) for f in os.listdir(folder_path)]
    
    # 过滤出图片文件
    image_paths = [f for f in all_files if os.path.isfile(f) and f.lower().endswith(supported_extensions)]
    
    return image_paths

def infer_and_save(config_path, checkpoint_path, img_path, save_path, save_result=True, infer_one_by_one=False, save_as_label_type=False):
    """
    使用模型加载权重推理img_path所有的图像，并将推理结果保存为.png图片于save_path路径下。
    
    参数:
    - config_path: 模型配置文件路径
    - checkpoint_path: 模型权重文件路径
    - img_path: 图像路径，可以是单个图像路径，也可以是图像路径的列表
    - save_path: 将保存的.png格式图片保存的位置
    - save_result: 默认为True，将推理的得到的result对象保存为.pkl文件于save_path路径下
    - infer_one_by_one: 默认为False，是否逐个推理路径列表中的每个路径
    - save_as_label_type: 默认为False，是否直接将numpy数组保存为.png图像
    
    返回值:
    - 推理的Result对象列表或单个Result对象
    """
    
    # 检查img_path是否为列表，如果不是，则转换为列表
    if not isinstance(img_path, list):
        img_path = [img_path]
    
    # 初始化模型
    model = init_model(config_path, checkpoint_path)
    
    results = []
    
    if infer_one_by_one:
        for img in img_path:
            # 推理单个图像
            result = inference_model(model, img)
            results.append(result)
    else:
        # 推理所有图像
        results = inference_model(model, img_path)
    
    # 检查结果是否为单个对象，如果是，则转换为列表
    if not isinstance(results, list):
        results = [results]
    
    # 创建保存路径目录（如果不存在）
    os.makedirs(save_path, exist_ok=True)
    
    for img, result in zip(img_path, results):
        # 将推理结果的张量转换为NumPy数组
        cpu_tensor = result.pred_sem_seg.data.cpu()
        numpy_array = cpu_tensor.numpy()
        
        # 获取原图像名并去除扩展名
        img_name = os.path.basename(img).rsplit('.', 1)[0]
        save_img_path = os.path.join(save_path, f"{img_name}.png")
        
        if save_as_label_type:
            # 直接将NumPy数组保存为.png格式
            Image.fromarray(numpy_array[0, :, :].astype(np.uint8)).save(save_img_path)
        else:
            # 将NumPy数组转换为PIL图像
            image = Image.fromarray(numpy_array[0, :, :].astype(np.uint8) * 255)
            # 保存图像为.png格式
            image.save(save_img_path)
        
        print(f"Segmentation result saved as {save_img_path}")
    
    if save_result:
        # 将结果保存为.pkl文件
        results_pkl_path = os.path.join(save_path, 'results.pkl')
        with open(results_pkl_path, 'wb') as f:
            pickle.dump(results, f)
        print(f"Results object saved as {results_pkl_path}")
    
    return results


In [37]:
# 示例调用
config_path = r"E:\github\MMLAB\mmsegmentation\my_mmseg_pretrain_model\Ulcer\Ours\hpc06091027_RFAinout_Dys_RatUlcer__mask2former_convnextv2-l.py"
checkpoint_path = r"E:\github\MMLAB\mmsegmentation\my_mmseg_pretrain_model\Ulcer\Ours\best_test_Ulcer_Dice_66.28_iter_19700.pth"
# img_path = [r'/mnt/workspace/project/MMLAB/work_dirs/inference/170719STZ01_06.jpg', r"/mnt/workspace/project/MMLAB/work_dirs/inference/230628ETT02_08.jpg"]
img_path = get_image_paths_from_folder(r"E:\github\MMLAB\mmsegmentation\my_mmseg_data\Rat_Ulcer\img_dir\test")

save_path = r'C:\Users\15154\Desktop\GEO'
infer_one_by_one = True
save_as_label_type = True
results = infer_and_save(config_path, checkpoint_path, img_path, save_path, infer_one_by_one=infer_one_by_one, save_as_label_type=save_as_label_type)

Loads checkpoint by local backend from path: E:\github\MMLAB\mmsegmentation\my_mmseg_pretrain_model\Ulcer\Ours\best_test_Ulcer_Dice_66.28_iter_19700.pth
Segmentation result saved as C:\Users\15154\Desktop\GEO\230308ETA03_01.png
Segmentation result saved as C:\Users\15154\Desktop\GEO\230308ETA03_02.png
Segmentation result saved as C:\Users\15154\Desktop\GEO\230308ETA03_03.png
Segmentation result saved as C:\Users\15154\Desktop\GEO\230308ETA03_04.png
Segmentation result saved as C:\Users\15154\Desktop\GEO\230308ETA03_05.png
Segmentation result saved as C:\Users\15154\Desktop\GEO\230308ETA03_06.png
Segmentation result saved as C:\Users\15154\Desktop\GEO\230308ETA03_07.png
Segmentation result saved as C:\Users\15154\Desktop\GEO\230308ETA05_01.png
Segmentation result saved as C:\Users\15154\Desktop\GEO\230308ETA05_02.png
Segmentation result saved as C:\Users\15154\Desktop\GEO\230308ETA05_03.png
Segmentation result saved as C:\Users\15154\Desktop\GEO\230308ETA05_04.png
Segmentation result sa

In [8]:
print(len(results))
print(results[0].__dict__)

247
{'_metainfo_fields': {'img_path', 'img_shape', 'scale_factor', 'ori_shape'}, '_data_fields': {'seg_logits', '_pred_sem_seg', 'pred_sem_seg', '_seg_logits'}, 'img_path': 'C:\\Users\\15154\\Desktop\\溃疡图像标注\\辅助标注的AI\\tissue推理的内容\\170719STZ01_06.jpg', 'img_shape': (640, 640), 'scale_factor': (0.5714285714285714, 0.5714285714285714), 'ori_shape': (1120, 1120), '_seg_logits': <PixelData(

    META INFORMATION

    DATA FIELDS
    data: tensor([[[9.9783e-01, 9.9783e-01, 9.9783e-01,  ..., 9.9632e-01,
                  9.9630e-01, 9.9630e-01],
                 [9.9783e-01, 9.9783e-01, 9.9783e-01,  ..., 9.9632e-01,
                  9.9630e-01, 9.9630e-01],
                 [9.9778e-01, 9.9778e-01, 9.9781e-01,  ..., 9.9633e-01,
                  9.9628e-01, 9.9628e-01],
                 ...,
                 [9.9594e-01, 9.9594e-01, 9.9600e-01,  ..., 9.9307e-01,
                  9.9299e-01, 9.9299e-01],
                 [9.9594e-01, 9.9594e-01, 9.9594e-01,  ..., 9.9295e-01,
                

In [31]:
print(results[0].pred_sem_seg.data[0].cpu().numpy())
print(results[0].pred_sem_seg.data[0].cpu().numpy().shape)
pred_mask = results[0].pred_sem_seg.data.cpu().numpy()
print(type(pred_mask))

[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]
(1120, 1120)
<class 'numpy.ndarray'>


In [30]:
np.unique(pred_mask)

array([0, 1, 2], dtype=int64)